In [31]:
import requests
import pandas as pd
from datetime import datetime
from textblob import TextBlob

In [38]:
def fetch_tesla_news(api_key, start_date, end_date):
    url = "https://api.marketaux.com/v1/news/all?symbols=TSLA&filter_entities=true&language=en&api_token=iy6rRX4oxFrouZocXr8JNpOzaxZLk3UvMfoMGxYs"  # Modify this based on the exact endpoint you need
    headers = {
        "x-api-key": api_key
    }
     # Since each page corresponds to a single request, limit the number of pages to 100
    for page in range(1, 101):  # start from page 1 to page 100
        params = {
            "tickers": "TSLA",
            "filter_entities": "true",
            "language": "en",
            "from": start_date,
            "to": end_date,
            "page": page
        }
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return pd.json_normalize(response.json()['data'])
    else:
        raise Exception(f"Failed to fetch data: {response.text}")

In [39]:
api_key = "iy6rRX4oxFrouZocXr8JNpOzaxZLk3UvMfoMGxYs"
tesla_news_df = fetch_tesla_news(api_key, "2017-01-01", datetime.now().strftime('%Y-%m-%d'))
print(tesla_news_df.head())

                                   uuid  \
0  daf76e3e-caea-4c92-a461-6b3132655788   
1  8dab10ca-5b23-465a-aa86-360bc987a774   
2  b8c381b9-4187-433e-ad15-cecc9d227b13   

                                               title  \
0  Stock market today: US futures climb as earnin...   
1  5 things to know before the stock market opens...   
2  Wall Street Breakfast Podcast: UNH: Personal D...   

                                         description  \
0  The wait for Tesla results is on as investors ...   
1  Here are the most important news items that in...   
2  UnitedHealth confirms personal data compromise...   

                                            keywords  \
0                                                      
1  Investment strategy, Economy, Markets, Busines...   
2                                                      

                                             snippet  \
0  US stocks climbed on Tuesday, on track for fur...   
1  In this article CPRI Follow your favo

In [30]:
# Clean text data, Check for any missing values or inconsistencies in the data
tesla_news_df['description'] = tesla_news_df['description'].apply(lambda x: x.lower().replace('[^\w\s]', ''))

# Check for any missing values
print(tesla_news_df.isnull().sum())


uuid               0
title              0
description        0
keywords           0
snippet            0
url                0
image_url          0
language           0
published_at       0
source             0
relevance_score    3
entities           0
similar            0
dtype: int64


In [32]:
# Sentiment analysis on descriptions
tesla_news_df['sentiment'] = tesla_news_df['description'].apply(lambda text: TextBlob(text).sentiment.polarity)

print(tesla_news_df[['title', 'sentiment']])


                                               title  sentiment
0          Wall Street Breakfast: What Moved Markets   0.197443
1  1 "Magnificent Seven" Stock With 1,234% Upside...   1.000000
2  Market Today: Tech Giants Reignite AI Craze, A...  -0.024242


In [35]:
# Example: Counting the number of articles per day
tesla_news_df['published_at'] = pd.to_datetime(tesla_news_df['published_at'])  # Convert to datetime
tesla_news_df['date'] = tesla_news_df['published_at'].dt.date
daily_news_count = tesla_news_df.groupby('date').size()


In [37]:
tesla_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   uuid             3 non-null      object             
 1   title            3 non-null      object             
 2   description      3 non-null      object             
 3   keywords         3 non-null      object             
 4   snippet          3 non-null      object             
 5   url              3 non-null      object             
 6   image_url        3 non-null      object             
 7   language         3 non-null      object             
 8   published_at     3 non-null      datetime64[ns, UTC]
 9   source           3 non-null      object             
 10  relevance_score  0 non-null      object             
 11  entities         3 non-null      object             
 12  similar          3 non-null      object             
 13  sentiment        3 non-n